In [ ]:
import pandas as pd
import random as rd
import random
from typing import Dict, List, Literal
from datasets import Dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss, TripletLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import InformationRetrievalEvaluator

from sentence_transformers import SentenceTransformerTrainer, SentenceTransformerTrainingArguments, SentenceTransformer
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers
from datasets import load_dataset, Dataset
import torch


train_data = pd.DataFrame('your train dataset ')
eval_data = pd.DataFrame('your  eval dataset ')

In [ ]:
torch.cuda.empty_cache()

link = "all-MiniLM-L6-v2"
device = torch.device('mps')
model = SentenceTransformer(link).to(device = device)
train_dataset = Dataset.from_pandas(train_data) # train_data is a private pandas dataFrame
eval_dataset = Dataset.from_pandas(eval_data)   # eval_data is a private pandas dataFrame

# The DataFrame should follow this column naming convention: "anchor", "positive", "negative_1", ..., "negative_{num_negatives}".
# Only "negative_1" will contain our full set of negative samples, while the other negative columns will be set to None.
# These remaining negative columns will be populated dynamically for each batch.

num_negatives = 5
def sample_positive_negatives(batch):
    anchors, positives, negatives = batch["anchor"], batch["positive"] , batch["negative_1"]

    sampled_batch = {
    'anchor': [],
    'positive': []
    }
    for i in range(1, num_negatives + 1):
        sampled_batch[f'negative_{i}'] = []

    for anchor, pos_list, neg_list in zip(anchors, positives, negatives):
        sampled_positive = random.choice(pos_list)  
        sampled_negatives = random.choices(neg_list, k=20) 
        sampled_batch["positive"].append(sampled_positive)
        sampled_batch["anchor"].append(anchor)

        for id in range(num_negatives) :
            sampled_batch[f"negative_{id+1}"].append(sampled_negatives[id])     

    return sampled_batch


# This allows the random sampling with the Trainer method :
train_dataset.set_transform(sample_positive_negatives)

loss = MultipleNegativesRankingLoss(model).to(device)

#  Specify training arguments
args = SentenceTransformerTrainingArguments(
    output_dir="models/mpnet-base-all-nli-triplet",
    num_train_epochs= 5,
    per_device_train_batch_size = 16, # 2
    per_device_eval_batch_size= 4, # 2
    warmup_ratio=0.1, 
    fp16=False,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=True ,   # Set to True if you have a GPU that supports BF16
    eval_strategy="steps",
    eval_steps=3000,
    save_strategy="steps",
    save_steps=1,
    save_total_limit=1,
    logging_steps=100,
    run_name="mpnet-base-all-nli-triplet" )

#  Create a trainer and train
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset , 
    loss=loss,
    eval_dataset = eval_dataset
)


trainer.train()